In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights,p_log, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items, size=n, replace=False, p=p_log)
    
    return recs.tolist()

In [6]:
def data_prep(df, func):
    # список уникальных товаров
    items = df.item_id.unique()

    items = np.array(items)

    # датафрейм с количеством каждого товара
    count_df = df.groupby('item_id').item_id.agg(['count'])

    # добавляем в датафрейм строчку с пересчитанными товарами в веса
    count_df['weight'] = func(count_df['count'] / count_df['count'].sum())

    # перводим items в датафрейм 
    items_df = pd.DataFrame(items, columns = ['item_id'])


    # создаём датафрейм item - вероятность с сохранением порядка item'ов
    probability_df = pd.merge(items_df, count_df, on='item_id', how='left')

    # заполняем пустые item's нулями
    probability_df.fillna(value=0, inplace=True)

    # берём столбец вероятностей и переводим его в numpy-array
    p = np.array(probability_df.to_numpy()[:, 2])

    # нормализация вероятностей для того чтобы они  были = 1
    p /= p.sum() 
    
    return items, p

In [7]:
%%time
items, p_log = data_prep(data_train, np.log)

result['weighted log rand rec'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, p_log, n=5))

result.head(3)

CPU times: user 2.04 s, sys: 152 ms, total: 2.2 s
Wall time: 2.03 s


,user_id,actual,weighted log rand rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1214177, 935833, 53337, 1227043, 477610]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[969727, 846176, 8158410, 13008098, 1917729]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6442764, 12731620, 1148215, 1079563, 13910853]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [8]:

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train_top5000 = data_train.loc[data_train['item_id'] != 6666]


/home/recpi/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [9]:
data_train_top5000.head(20)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
5,2375,26984851516,1,826249,2,1.98,364,-0.60,1642,1,0.0,0.0
7,2375,26984851516,1,1085983,1,2.99,364,-0.40,1642,1,0.0,0.0
11,1364,26984896261,1,842930,1,2.19,31742,0.00,1520,1,0.0,0.0
15,1364,26984896261,1,981760,1,0.60,31742,-0.79,1520,1,0.0,0.0
16,1130,26984905972,1,833715,2,0.34,31642,-0.32,1340,1,0.0,0.0


In [10]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [11]:
%%time

items = data_train_top5000.item_id.unique()

result['random_recommendation_top5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(10)

CPU times: user 214 ms, sys: 391 µs, total: 214 ms
Wall time: 212 ms


,user_id,actual,weighted log rand rec,random_recommendation_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1214177, 935833, 53337, 1227043, 477610]","[1052335, 9296847, 9677949, 917742, 7440663]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[969727, 846176, 8158410, 13008098, 1917729]","[1098910, 868683, 886787, 1093730, 993685]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6442764, 12731620, 1148215, 1079563, 13910853]","[14050461, 6424472, 5571248, 1075786, 921614]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[13776390, 6772967, 2058885, 881529, 5716558]","[1101449, 877935, 1112238, 6632283, 967461]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[883474, 7406726, 1029549, 6633344, 2926952]","[887661, 898869, 1055412, 1098066, 907014]"
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8...","[6904451, 1090286, 1005172, 10456500, 7166862]","[9677236, 838097, 5566235, 904574, 904422]"
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884...","[1139036, 1028438, 5589557, 852773, 889994]","[996009, 831628, 12781986, 1089074, 12987975]"
7,14,"[840601, 867293, 933067, 951590, 952408, 96569...","[914561, 9928380, 2763199, 13159598, 1593899]","[12425418, 964327, 939546, 922561, 920308]"
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13...","[6391403, 5588798, 870791, 932135, 859425]","[1119405, 993638, 912280, 9707240, 6463742]"
9,16,"[1062973, 1082185, 13007710]","[1027099, 16223139, 15596011, 934618, 1033562]","[855488, 1023466, 16809471, 1046243, 986947]"


In [12]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [13]:
%%time

popular_recs = popularity_recommendation(data_train_top5000, n=5)

result['popular_recommendation_top5000'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 34.2 ms, sys: 0 ns, total: 34.2 ms
Wall time: 33 ms


,user_id,actual,weighted log rand rec,random_recommendation_top5000,popular_recommendation_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1214177, 935833, 53337, 1227043, 477610]","[1052335, 9296847, 9677949, 917742, 7440663]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[969727, 846176, 8158410, 13008098, 1917729]","[1098910, 868683, 886787, 1093730, 993685]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [14]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(items_weights.item_id, size=n, replace=False, p=items_weights.weight)
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [15]:
items_weights = data_train_top5000.groupby('item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value': 'weight'})
items_weights.loc[items_weights['weight'] < 1, 'weight'] = 1
items_weights['weight'] = np.log(items_weights['weight'])
items_weights['weight'] = items_weights['weight'] / items_weights['weight'].sum()

result['weighted_random_top5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)


,user_id,actual,weighted log rand rec,random_recommendation_top5000,popular_recommendation_top5000,weighted_random_top5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1214177, 935833, 53337, 1227043, 477610]","[1052335, 9296847, 9677949, 917742, 7440663]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6442594, 825249, 1080414, 1009770, 1094371]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[969727, 846176, 8158410, 13008098, 1917729]","[1098910, 868683, 886787, 1093730, 993685]","[6534178, 6533889, 1029743, 6534166, 1082185]","[7410013, 1018007, 834491, 1139651, 950098]"


In [19]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")


1.0:actual
0.0006:weighted log rand rec
0.0062:random_recommendation_top5000
0.1552:popular_recommendation_top5000
0.0077:weighted_random_top5000


#### Пробуем улучшить разные варианты ItemItemRecommender

In [20]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [21]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [22]:
metrics_list = []
for i in range(10):
    model = ItemItemRecommender(K=i+1, num_threads=4) # K - кол-во билжайших соседей


    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    tmp_df = result[['actual']]

    tmp_df['itemitem'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])
    
    metrics_list.append(tmp_df.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean())

for idx, itm in enumerate(metrics_list):
    print(f'precision_at_k for K={idx+1}: {itm}')

<ipython-input-22-ebea88bafcfa>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['itemitem'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in



precision_at_k for K=1: 0.19229513548808072
precision_at_k for K=2: 0.1919849820437451
precision_at_k for K=3: 0.1860920666013684
precision_at_k for K=4: 0.14485798237022346
precision_at_k for K=5: 0.13682664054848048
precision_at_k for K=6: 0.14211557296767716
precision_at_k for K=7: 0.14495592556317166
precision_at_k for K=8: 0.14720861900097773
precision_at_k for K=9: 0.14848188050930292
precision_at_k for K=10: 0.15093046033300497


лучшая метрика достигается при K=2